In [19]:
import os
import requests
from bs4 import BeautifulSoup
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import time

# URL de la página web a scrapear
url = "https://iol.invertironline.com/titulo/cotizacion/BCBA/AL30D/BONO-REP.-ARGENTINA-USD-STEP-UP-2030/"

# Ruta del archivo de Excel
ruta_excel = "/Users/mrz/Documents/jupyter/anker/precio.xlsx"


# Crear el archivo de Excel si no existe
if not os.path.exists(ruta_excel):
    wb = Workbook()
    ws = wb.active
    ws.title = "Precio"
    ws.column_dimensions[get_column_letter(1)].width = 20
    ws['A1'] = "Último Precio"
    ws['A1'].alignment = Alignment(horizontal='center')
    wb.save(ruta_excel)

# Cargar el archivo de Excel existente
wb = load_workbook(ruta_excel)
ws = wb.active

# Definir la función para actualizar y guardar el precio
def actualizar_precio():
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        elemento_precio = soup.find(attrs={"data-field": "UltimoPrecio"})
        
        if elemento_precio is not None:
            ultimo_precio = elemento_precio.text.strip()
            
            # Obtener la celda siguiente vacía en la columna A
            celda_vacia = ws.cell(row=ws.max_row + 1, column=1)
            celda_vacia.value = ultimo_precio
            
            # Guardar el archivo de Excel
            wb.save(ruta_excel)
        else:
            print("No se encontró el dato 'UltimoPrecio' en la página.")
    else:
        print("Error al obtener la página:", response.status_code)

# Actualizar y guardar el precio cada minuto
while True:
    actualizar_precio()
    time.sleep(60)  # Esperar 60 segundos


KeyboardInterrupt: 

Note: you may need to restart the kernel to use updated packages.


In [24]:
import requests
import time
from bs4 import BeautifulSoup
from openpyxl import Workbook
from openpyxl.utils import get_column_letter

# URL de la página web a scrapear
url = "https://www.portfoliopersonal.com/Cotizaciones/Bonos"

# Ruta del archivo de Excel
ruta_excel = "/Users/mrz/Documents/jupyter/anker/precio.xlsx"

while True:
    # Realizar la solicitud HTTP GET a la página
    response = requests.get(url)

    # Comprobar el estado de la respuesta
    if response.status_code == 200:
        # Crear un objeto BeautifulSoup con el contenido HTML de la página
        soup = BeautifulSoup(response.content, "html.parser")

        # Encontrar la tabla de cotizaciones
        tabla_cotizaciones = soup.find("table", class_="table")

        if tabla_cotizaciones is not None:
            # Crear el archivo de Excel
            wb = Workbook()
            ws = wb.active
            ws.title = "Cotizaciones"

            # Obtener las filas de la tabla
            filas = tabla_cotizaciones.find_all("tr")

            # Iterar sobre las filas y guardar los datos en el archivo de Excel
            for fila_index, fila in enumerate(filas):
                celdas = fila.find_all("td")
                for indice, celda in enumerate(celdas):
                    columna = get_column_letter(indice + 1)
                    ws[columna + str(fila_index + 1)] = celda.text.strip()

            # Guardar el archivo de Excel
            wb.save(ruta_excel)

            print("Tabla de cotizaciones guardada en el archivo:", ruta_excel)
        else:
            print("No se encontró la tabla de cotizaciones en la página.")
    else:
        print("Error al obtener la página:", response.status_code)

    # Esperar 20 minutos antes de la siguiente ejecución
    time.sleep(20 * 60)  # 20 minutos en segundos (20 * 60)


Tabla de cotizaciones guardada en el archivo: /Users/mrz/Documents/jupyter/anker/precio.xlsx
